In [1]:
import time
from typing import Dict, List, Tuple, Optional

# --- 1. Graph Construction ---

def build_MT3n_graph(n: int) -> Tuple[Dict[int, List[int]], List[int]]:
    """
    Builds the Adjacency List for the Mongolian Tent Graph MT_{3,n} in O(n) time.

    The vertex ordering is critical for the labeling algorithm:
    - index 0 = apex z
    - indices 1..n = top row (v_1,1 to v_1,n)
    - indices n+1..2n = middle row (v_2,1 to v_2,n)
    - indices 2n+1..3n = bottom row (v_3,1 to v_3,n)

    Total vertices V = 3n + 1.

    Args:
        n: The width parameter (n >= 2, but constraint is 5 <= n <= 50).

    Returns:
        A tuple: (Adjacency List, Fixed Vertex Order List)
    """
    if n < 1:
        raise ValueError("Parameter n must be greater than or equal to 1.")

    V = 3 * n + 1
    adj: Dict[int, List[int]] = {i: [] for i in range(V)}

    # Fixed vertex ordering for the labeling algorithm
    order = list(range(V))

    # 1. Apex edges (0 to 1..n)
    for i in range(1, n + 1):
        adj[0].append(i)
        adj[i].append(0)

    # Helper function for adding path edges (horizontal)
    def add_horizontal(start: int, end: int):
        for i in range(start, end):
            adj[i].append(i + 1)
            adj[i + 1].append(i)

    # 2. Horizontal edges in each row
    add_horizontal(1, n)            # Top row (1..n)
    add_horizontal(n + 1, 2 * n)    # Middle row (n+1..2n)
    add_horizontal(2 * n + 1, 3 * n)  # Bottom row (2n+1..3n)

    # 3. Vertical rungs (connections between rows)
    for i in range(1, n + 1):
        top = i
        mid = n + i
        bottom = 2 * n + i

        # Top <-> Middle
        adj[top].append(mid)
        adj[mid].append(top)

        # Middle <-> Bottom
        adj[mid].append(bottom)
        adj[bottom].append(mid)

    return adj, order

# --- 2. Labeling Algorithm (Backtracking DFS) ---

def solve_MT3n_labeling(n: int) -> Tuple[int, Optional[List[int]]]:
    """
    Finds the minimum label set size k (k=3n-1, k0+1, ...) that permits
    a valid labeling where:
    1. Vertex labels l(v) are unique and in {0, ..., k}.
    2. Edge weights w(u,v) = l(u) + l(v) are unique, non-zero, and <= 2k.

    Args:
        n: The width parameter of the MT_3,n graph.

    Returns:
        A tuple: (Minimum k found, List of V labels or None if failed)
    """
    adj, order = build_MT3n_graph(n)
    V = 3 * n + 1

    # Initial lower bound k0 = 3n - 1
    k = 3 * n - 1

    print(f"Graph MT_3,{n}: V={V} vertices.")
    print(f"Starting search for minimum k (label set size) from k={k}...")

    while True:
        start_time = time.time()

        # Initialization for the current k
        labels = [-1] * V  # Stores l(v), -1 means unassigned
        usedLabel = [False] * (k + 1)  # Tracks unique vertex labels used (0 to k)
        # Tracks unique edge weights used (1 to 2k). Index 0 is unused (weight=0 is forbidden)
        usedWeight = [False] * (2 * k + 1)

        print(f"\n--- Testing k = {k} ---")

        def backtrack(idx: int) -> bool:
            """
            Recursive DFS to assign label to vertex order[idx].
            """
            if idx == V:
                # Success: All vertices assigned valid labels
                return True

            u = order[idx]

            # Iterate through all available labels (0 to k)
            for current_label in range(k + 1):

                # Constraint 1: Vertex labels must be unique
                if usedLabel[current_label]:
                    continue

                ok = True
                new_weights = []

                # Constraint 2: Check edge weight uniqueness with ALREADY LABELED neighbors
                for v in adj[u]:
                    if labels[v] != -1:
                        # Neighbor 'v' is already labeled
                        w = current_label + labels[v]

                        # Constraint checks: w must be positive, <= 2k, and unused
                        if w == 0 or w > 2 * k or usedWeight[w]:
                            ok = False
                            break
                        new_weights.append(w)

                if not ok:
                    continue  # Try next label

                # --- Assignment (Pruning passed) ---
                labels[u] = current_label
                usedLabel[current_label] = True
                for w in new_weights:
                    usedWeight[w] = True

                # Recurse to next vertex
                if backtrack(idx + 1):
                    return True

                # --- Backtrack (Undo assignment) ---
                labels[u] = -1
                usedLabel[current_label] = False
                for w in new_weights:
                    usedWeight[w] = False

            return False

        # Start the recursive search
        if backtrack(0):
            runtime = time.time() - start_time
            print(f"Success! Minimum k = {k}")
            print(f"Runtime for k={k}: {runtime:.4f} seconds.")
            return k, labels

        runtime = time.time() - start_time
        print(f"Failed for k={k}. Runtime: {runtime:.4f} seconds.")

        # If failed for current k, increment k and try again
        k += 1

        # Safety break for potentially infinite loop/long runtimes on large n
        if k > (3 * n + 5):
            print("Warning: Search limit reached without finding a solution.")
            return -1, None


# --- 3. Example Execution (Suitable for Notebook) ---

if __name__ == "__main__":

    # NOTE: Due to exponential complexity (k^(3n+1)),
    # use small values of n (e.g., n=1, 2, or max 3) for quick results.
    # Larger values of n (e.g., n=5) will take an extremely long time.

    N_TEST = 2
    # V = 3(2) + 1 = 7 vertices. k0 = 3(2) - 1 = 5.

    print(f"# --- Running MT_3,n Labeling for n={N_TEST} ---")

    try:
        min_k, result_labels = solve_MT3n_labeling(N_TEST)

        if result_labels is not None:
            print("\n## ✨ Final Result ✨ ##")
            print(f"Graph: MT_3,{N_TEST}")
            print(f"Minimum Label Set Size k: {min_k}")
            print(f"Vertex Labels (0..{min_k}): {result_labels}")

            # Additional verification: Check vertex labels and edge weights
            adj, _ = build_MT3n_graph(N_TEST)
            edge_weights = set()

            for u in range(len(result_labels)):
                for v in adj[u]:
                    if u < v: # Check each edge once
                        w = result_labels[u] + result_labels[v]
                        if w in edge_weights:
                            print(f"\nERROR: Duplicate weight {w} found!")
                        edge_weights.add(w)

            print(f"Total Unique Edge Weights: {len(edge_weights)}")
            print(f"Required Unique Edge Weights: {(len(adj) // 2)}") # V-1 for graceful?

            # For a proper graceful labeling, edges = V-1, weights = 1..V-1 or 1..E
            # For this specific constraint: V unique labels in {0..k}, E unique weights in {1..2k}
            # Max possible unique labels: k+1
            # Number of edges E = 7n - 3 = 11 for n=2.
            print(f"Number of Edges E: {(sum(len(adj[i]) for i in adj) // 2)}")

        else:
            print("\nLabeling failed to find a solution within the search range.")

    except ValueError as e:
        print(f"\nError: {e}")

# --- Running MT_3,n Labeling for n=2 ---
Graph MT_3,2: V=7 vertices.
Starting search for minimum k (label set size) from k=5...

--- Testing k = 5 ---
Failed for k=5. Runtime: 0.0013 seconds.

--- Testing k = 6 ---
Success! Minimum k = 6
Runtime for k=6: 0.0000 seconds.

## ✨ Final Result ✨ ##
Graph: MT_3,2
Minimum Label Set Size k: 6
Vertex Labels (0..6): [0, 1, 2, 3, 4, 5, 6]
Total Unique Edge Weights: 9
Required Unique Edge Weights: 3
Number of Edges E: 9
